In [6]:
%run main.py

INFO:scene_setup:Starting scene setup with ray tracing...
DEBUG:scene_setup:Debugging Base station state:


XLA can lead to reduced numerical precision. Use with care.


DEBUG:scene_setup:- Position: [10.  10.   4.5]
DEBUG:scene_setup:- Orientation: [  0.   0. -90.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x0000026723944550>
DEBUG:scene_setup:Debugging RIS state:
DEBUG:scene_setup:- Position: [10.  19.5  2.5]
DEBUG:scene_setup:- Orientation: [  0. -90.   0.]
DEBUG:scene_setup:Debugging AGV_0 state:
DEBUG:scene_setup:- Position: [12.   5.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x00000266CAA5E510>
DEBUG:scene_setup:Debugging AGV_1 state:
DEBUG:scene_setup:- Position: [ 8.  15.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x00000266CAA2ED10>
INFO:scene_setup:Scene setup completed successfully
INFO:scene_setup:- Transmitters: 7
INFO:scene_setup:- Receivers: 2
INFO:scene_setup:- RIS: 1
INFO:scene_setup:- Objects: 0
INFO:scene_setup:- Ray tr

Generating channel data...
Generating channel data...
Channel data generation completed
Channel data saved to: C:\Users\snatanzi\Desktop\Causal-XAI\src\results\channel_data.h5

Channel Data Statistics:
Channel matrices shape: (1, 2, 1, 7, 128, 1, 1024)
Path delays shape: (1, 2, 7, 2)
Number of receivers: 2
Execution completed


In [7]:
# After running main.py, add these lines to visualize the scene
scene = setup_scene(config)  # Get your scene

# Preview the scene interactively (in Jupyter)
scene.preview()

# Or render with specific camera angles
scene.render(camera="Camera.001")  # Try different camera numbers

# To visualize with paths
paths = scene.compute_paths()
scene.preview(paths=paths)

# Save the scene for later use
scene.save('factory_scene.xml')

NameError: name 'config' is not defined